In [1]:
library(forecast)
library(M4comp2018)
library(dplyr)

Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [2]:
etsl=load('year_ets_datalist.RData')
ets_datalist <- eval(parse(text = etsl))
thetafl=load('year_thetaf_datalist.RData')
thetaf_datalist <- eval(parse(text = thetafl))

In [3]:
ets_predh=ets_datalist[[2]]
thetaf_predh=thetaf_datalist[[2]]

In [4]:
ets_pred_res=ets_datalist[[1]]
thetaf_pred_res=thetaf_datalist[[1]]

In [5]:
etsoptl=load('yearly_ets_opt_pre_result.RData')
ets_opt_pre_list<- eval(parse(text = etsoptl))
thetafoptl=load('yearly_thetaf_opt_pre_result.RData')
thetaf_opt_pre_list<- eval(parse(text = thetafoptl))

In [6]:
time_matrix <- matrix(0,ncol = 3, nrow =3)
colnames(time_matrix) <- c("user_time", "system_time", "elapsed_time")
start_time = Sys.time()

In [7]:
ets_pr_min=ets_opt_pre_list[[1]]
ets_pr_mean=ets_opt_pre_list[[2]]
thetaf_pr_min=thetaf_opt_pre_list[[1]]
thetaf_pr_mean=thetaf_opt_pre_list[[2]]

In [8]:
head(ets_pr_min)

xgbclsmin,xgbregmin,lgbclsmin,lgbregmin
4,1.817922,1,1.784656
4,2.699616,4,2.279139
1,1.139816,3,2.016433
4,3.583172,2,2.646442
4,2.611977,2,3.115168
4,4.287028,4,3.684896


In [9]:
ets_pr_min=ets_pr_min+1
ets_pr_mean=ets_pr_mean+1
thetaf_pr_min=thetaf_pr_min+1
thetaf_pr_mean=thetaf_pr_mean+1


In [10]:
head(ets_pr_min)

xgbclsmin,xgbregmin,lgbclsmin,lgbregmin
5,2.817922,2,2.784656
5,3.699616,5,3.279139
2,2.139816,4,3.016433
5,4.583172,3,3.646442
5,3.611977,3,4.115168
5,5.287028,5,4.684896


In [11]:
round5=function(x)
    {
    l=round(x)
    for(i in 1:length(l))
    {
    if(l[i]>5)
        {
        l[i]=5
    }
    if(l[i]<1)
    {
        l[i]=1
    }
}
    return(l)
}

In [12]:
ets_pr_min[,2]=round5(ets_pr_min[,2])
ets_pr_min[,4]=round5(ets_pr_min[,4])

In [13]:
ets_pr_mean[,2]=round5(ets_pr_mean[,2])
ets_pr_mean[,4]=round5(ets_pr_mean[,4])

In [14]:
thetaf_pr_min[,2]=round5(thetaf_pr_min[,2])
thetaf_pr_min[,4]=round5(thetaf_pr_min[,4])

In [15]:
thetaf_pr_mean[,2]=round5(thetaf_pr_mean[,2])
thetaf_pr_mean[,4]=round5(thetaf_pr_mean[,4])

In [16]:
head(ets_pr_min)

xgbclsmin,xgbregmin,lgbclsmin,lgbregmin
5,3,2,3
5,4,5,3
2,2,4,3
5,5,3,4
5,4,3,4
5,5,5,5


In [17]:
Yearly_M4 <- Filter(function(l) l$period == "Yearly", M4)

In [18]:
data=Yearly_M4

In [19]:
datalength=length(data)
h=data[[1]]$h
m=10
n=4
freq=frequency(data[[1]]$x)

In [20]:
realxx=matrix(0,datalength,h)
for(i in 1:datalength){
    realxx[i,]=data[[i]]$xx
}

In [21]:
length5=matrix(0,datalength,m)
for(k in 1:datalength)
{
y = data[[k]]$x
y_l=length(y)
loc = 1:length(y)
loc_m = as.integer(loc*m/length(y))
filt_d = data.frame(y,loc_m)
filt_0=filter(filt_d ,loc_m==0)
m_l=count(filt_0)
m_l=as.integer(m_l)
length5[k,]=c(0,1*m_l,2*m_l,3*m_l,4*m_l)
    }

In [22]:
end_time = Sys.time()
time_matrix[1,]=end_time-start_time

In [23]:
end_time-start_time

Time difference of 1.349246 mins

## ETS

In [24]:
start_time = Sys.time()

In [25]:
set.seed(100)
index = sample(2,length(data),replace = TRUE,prob=c(0.7,0.3))
testindex=c(1:length(data))[index==2]

In [26]:
meanpre=function(pred_array,opt_pr,h,testindex)
    {
    data_length=length(testindex)
    meanpreh=matrix(0,data_length,h)
    for (i in 1:data_length){
    index=testindex[i]
    loc=opt_pr[index]
    predh=pred_array[index,loc,,]
    predh=unique(predh)
    meanpreh[i,]=apply(predh,2,mean)
}
    return(meanpreh)
}

In [27]:
predres=function(pred_matrix,data,opt_pr,lengthmatrix,h,testindex)
    {
    data_length=length(testindex)
    pred_res=matrix(0,data_length,6)
    model=ets(data[[1]]$x)
    fore_l=forecast(model,h=h)
    freq=frequency(data[[1]]$x)
    for (i in 1:data_length){
    index=testindex[i]
    loc=opt_pr[index]
    start=lengthmatrix[i,loc]
    y_all=data[[index]]$x
    y=y_all[start:length(y_all)]
    fore_l$x=ts(y,frequency=freq,end=end(y_all))
    real=data[[index]]$xx
    fore_l$mean=ts(pred_matrix[i,],start=start(real),frequency=freq)
    res=accuracy(fore_l,real)
    pred_res[i,]=res[2,1:6]
    }
    return(pred_res)
    }

In [28]:
predres_mul=function(pred_matrix,data,opt_pr_mul,lengthmatrix,h,testindex)
    {
    data_length=length(testindex)
    pred_res=matrix(0,data_length,6)
    model=ets(data[[1]]$x)
    fore_l=forecast(model,h=h)
    freq=frequency(data[[1]]$x)
    for (i in 1:data_length){
    index=testindex[i]
    loc=min(opt_pr_mul[index,])
    start=lengthmatrix[i,loc]
    y_all=data[[index]]$x
    y=y_all[start:length(y_all)]
    fore_l$x=ts(y,frequency=freq,end=end(y_all))
    real=data[[index]]$xx
    fore_l$mean=ts(pred_matrix[i,],start=start(real),frequency=freq)
    res=accuracy(fore_l,real)
    pred_res[i,]=res[2,1:6]
    }
    return(pred_res)
    }

In [29]:
etsxgbcls_min=meanpre(ets_predh,ets_pr_min[,1],h,testindex)
etsxgbreg_min=meanpre(ets_predh,ets_pr_min[,2],h,testindex)
etslgbcls_min=meanpre(ets_predh,ets_pr_min[,3],h,testindex)
etslgbreg_min=meanpre(ets_predh,ets_pr_min[,4],h,testindex)

In [30]:
etsxgbcls_mean=meanpre(ets_predh,ets_pr_mean[,1],h,testindex)
etsxgbreg_mean=meanpre(ets_predh,ets_pr_mean[,2],h,testindex)
etslgbcls_mean=meanpre(ets_predh,ets_pr_mean[,3],h,testindex)
etslgbreg_mean=meanpre(ets_predh,ets_pr_mean[,4],h,testindex)

### The error of each improved prediction result

In [31]:
lengthtest=length5[testindex,]

In [32]:
etsminxgbclsres=predres(etsxgbcls_min,data,ets_pr_min[,1],lengthtest,h,testindex)
etsminxgbregres=predres(etsxgbreg_min,data,ets_pr_min[,2],lengthtest,h,testindex)
etsminlgbclsres=predres(etslgbcls_min,data,ets_pr_min[,3],lengthtest,h,testindex)
etsminlgbregres=predres(etslgbreg_min,data,ets_pr_min[,4],lengthtest,h,testindex)

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

...

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

In [33]:
etsmeanxgbclsres=predres(etsxgbcls_mean,data,ets_pr_mean[,1],lengthtest,h,testindex)
etsmeanxgbregres=predres(etsxgbreg_mean,data,ets_pr_mean[,2],lengthtest,h,testindex)
etsmeanlgbclsres=predres(etslgbcls_mean,data,ets_pr_mean[,3],lengthtest,h,testindex)
etsmeanlgbregres=predres(etslgbreg_mean,data,ets_pr_mean[,4],lengthtest,h,testindex)

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

...

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

In [34]:
end_time = Sys.time()
time_matrix[2,]=end_time-start_time

### The results of the two methods of determining labels are averaged respectively

In [35]:
etsallmin=(etsxgbcls_min+etslgbcls_min+etsxgbreg_min+etslgbreg_min)/4
etsallmean=(etsxgbcls_mean+etslgbcls_mean+etsxgbreg_mean+etslgbreg_mean)/4

In [36]:
etsallminres=predres_mul(etsallmin,data,ets_pr_min,lengthtest,h,testindex)
etsallmeanres=predres_mul(etsallmean,data,ets_pr_mean,lengthtest,h,testindex)

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

### Under the above classification, the improved forecast results of the classification and regression models are averaged respectively

In [37]:
etsclsmin=(etsxgbcls_min+etslgbcls_min)/2
etsregmin=(etsxgbreg_min+etslgbreg_min)/2
etsclsmean=(etsxgbcls_mean+etslgbcls_mean)/2
etsregmean=(etsxgbreg_mean+etslgbreg_mean)/2

In [38]:
etsclsminres=predres_mul(etsclsmin,data,ets_pr_min[,c(1,3)],lengthtest,h,testindex)
etsregminres=predres_mul(etsregmin,data,ets_pr_min[,c(2,4)],lengthtest,h,testindex)
etsclsmeanres=predres_mul(etsclsmean,data,ets_pr_mean[,c(1,3)],lengthtest,h,testindex)
etsregmeanres=predres_mul(etsregmean,data,ets_pr_mean[,c(2,4)],lengthtest,h,testindex)

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

### Random slice

In [39]:
etspred_change<- aperm(ets_predh[1:datalength,1:m,1:n,1:h], c(1, 3, 2, 4))  
# 使用array函数将其转换为新的形状
etspred_change <- array(etspred_change, dim = c(datalength, m*n, h))

In [40]:
model=ets(data[[1]]$x)
fore_l=forecast(model,h=h)
etsrandomres=matrix(0,datalength,6)
for (i in 1:datalength){
    random=sample(m*n,n)
    minloc=min(random)
    # 使用cut函数将数据分割为区间
    loc <- cut(minloc, breaks =seq(0,m*n,n), labels = FALSE)
    loc<-as.numeric(loc)
    etsrandompred=apply(etspred_change[i,random,],2,mean)
    start=length5[i,loc]
    y_all=data[[i]]$x
    y=y_all[start:length(y_all)]
    fore_l$x=ts(y,frequency=freq,end=end(y_all))
    real=data[[i]]$xx
    fore_l$mean=ts(etsrandompred,start=start(real),frequency=freq)
    res=accuracy(fore_l,real)
    etsrandomres[i,]=res[2,1:6]
    }

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

In [41]:
mean1=function(x)
    {
# 使用is.finite函数检查哪些值是有限的
finite_values <- x[is.finite(x)]
# 计算有限值的平均值
d=mean(finite_values, na.rm = TRUE)
return(d)
}

In [42]:
print(apply(ets_pred_res[testindex,1,1,],2,mean1))
print(apply(etsminxgbclsres,2,mean1))
print(apply(etsminxgbregres,2,mean1))
print(apply(etsminlgbclsres,2,mean1))
print(apply(etsminlgbregres,2,mean1))

[1]  216.284637 1056.435200  923.917192   -1.948543   18.063561    3.443647
[1]  371.1853980 1046.5594112  911.7943733   -0.1092825   17.6814929
[6]    2.7308075
[1]  321.343595 1028.649151  897.829086   -1.204191   17.426648    2.715083
[1]  300.019683 1028.812332  897.004236   -1.194031   17.387994    3.005645
[1]  346.7658009 1014.1257430  884.9012405   -0.6252169   16.8421275
[6]    2.6571321


In [43]:
print(apply(etsmeanxgbclsres,2,mean1))
print(apply(etsmeanxgbregres,2,mean1))
print(apply(etsmeanlgbclsres,2,mean1))
print(apply(etsmeanlgbregres,2,mean1))

[1]  265.241002 1009.134218  880.086674   -1.694540   17.127970    2.600923
[1]  304.087042 1006.555503  878.325232   -1.137550   16.831074    2.723801
[1]  290.373218 1029.925135  898.460667   -1.553873   17.597015    2.987274
[1]  309.8603640 1003.9531603  876.1339011   -0.9937028   16.7086347
[6]    2.7205099


In [44]:
print(apply(etsallminres,2,mean1))
print(apply(etsallmeanres,2,mean1))

[1] 334.8286193 997.2745652 867.8468842  -0.7831805  16.8010017   2.8599470
[1] 292.390407 984.328229 857.192592  -1.344916  16.625815   2.854181


In [45]:
print(apply(etsclsminres,2,mean1))
print(apply(etsregminres,2,mean1))
print(apply(etsclsmeanres,2,mean1))
print(apply(etsregmeanres,2,mean1))

[1]  335.6025408 1010.8745025  879.4644944   -0.6516569   17.0990217
[6]    2.8849455
[1]  334.0546979 1009.9978468  880.6798883   -0.9147041   16.9520882
[6]    2.6929215
[1] 277.807110 994.145886 865.690607  -1.624206  16.966122   2.835298
[1] 306.973703 998.458605 870.821762  -1.065626  16.673760   2.728826


In [46]:
print(apply(etsrandomres[testindex,],2,mean))

[1] 295.124734 998.156171 868.918207  -1.335380  16.795238   3.136813


In [47]:
etsall=matrix(0,16,6)
etsall[1,]=apply(ets_pred_res[testindex,1,1,],2,mean1)[1:6]
etsall[2,]=apply(etsminxgbclsres,2,mean1)
etsall[3,]=apply(etsminxgbregres,2,mean1)
etsall[4,]=apply(etsminlgbclsres,2,mean1)
etsall[5,]=apply(etsminlgbregres,2,mean1)
etsall[6,]=apply(etsmeanxgbclsres,2,mean1)
etsall[7,]=apply(etsmeanxgbregres,2,mean1)
etsall[8,]=apply(etsmeanlgbclsres,2,mean1)
etsall[9,]=apply(etsmeanlgbregres,2,mean1)
etsall[10,]=apply(etsallminres,2,mean1)
etsall[11,]=apply(etsallmeanres,2,mean1)
etsall[12,]=apply(etsclsminres,2,mean1)
etsall[13,]=apply(etsregminres,2,mean1)
etsall[14,]=apply(etsclsmeanres,2,mean1)
etsall[15,]=apply(etsregmeanres,2,mean1)
etsall[16,]=apply(etsrandomres[testindex,],2,mean)

In [48]:
etsall

216.2846,1056.4352,923.9172,-1.9485431,18.06356,3.443647
371.1854,1046.5594,911.7944,-0.1092825,17.68149,2.730808
321.3436,1028.6492,897.8291,-1.2041913,17.42665,2.715083
300.0197,1028.8123,897.0042,-1.1940313,17.38799,3.005645
346.7658,1014.1257,884.9012,-0.6252169,16.84213,2.657132
265.2410,1009.1342,880.0867,-1.6945396,17.12797,2.600923
304.0870,1006.5555,878.3252,-1.1375500,16.83107,2.723801
290.3732,1029.9251,898.4607,-1.5538732,17.59702,2.987274
309.8604,1003.9532,876.1339,-0.9937028,16.70863,2.720510
334.8286,997.2746,867.8469,-0.7831805,16.80100,2.859947
292.3904,984.3282,857.1926,-1.3449164,16.62582,2.854181


In [49]:
write.csv(etsall,'ets_Yearly_final_res.csv')

## thetaf

### The error of each improved prediction result

In [50]:
start_time = Sys.time()

In [51]:
thetafxgbcls_min=meanpre(thetaf_predh,thetaf_pr_min[,1],h,testindex)
thetafxgbreg_min=meanpre(thetaf_predh,thetaf_pr_min[,2],h,testindex)
thetaflgbcls_min=meanpre(thetaf_predh,thetaf_pr_min[,3],h,testindex)
thetaflgbreg_min=meanpre(thetaf_predh,thetaf_pr_min[,4],h,testindex)

In [52]:
thetafxgbcls_mean=meanpre(thetaf_predh,thetaf_pr_mean[,1],h,testindex)
thetafxgbreg_mean=meanpre(thetaf_predh,thetaf_pr_mean[,2],h,testindex)
thetaflgbcls_mean=meanpre(thetaf_predh,thetaf_pr_mean[,3],h,testindex)
thetaflgbreg_mean=meanpre(thetaf_predh,thetaf_pr_mean[,4],h,testindex)

In [53]:
thetafminxgbclsres=predres(thetafxgbcls_min,data,thetaf_pr_min[,1],lengthtest,h,testindex)
thetafminxgbregres=predres(thetafxgbreg_min,data,thetaf_pr_min[,2],lengthtest,h,testindex)
thetafminlgbclsres=predres(thetaflgbcls_min,data,thetaf_pr_min[,3],lengthtest,h,testindex)
thetafminlgbregres=predres(thetaflgbreg_min,data,thetaf_pr_min[,4],lengthtest,h,testindex)

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

...

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

In [54]:
thetafmeanxgbclsres=predres(thetafxgbcls_mean,data,thetaf_pr_mean[,1],lengthtest,h,testindex)
thetafmeanxgbregres=predres(thetafxgbreg_mean,data,thetaf_pr_mean[,2],lengthtest,h,testindex)
thetafmeanlgbclsres=predres(thetaflgbcls_mean,data,thetaf_pr_mean[,3],lengthtest,h,testindex)
thetafmeanlgbregres=predres(thetaflgbreg_mean,data,thetaf_pr_mean[,4],lengthtest,h,testindex)

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

...

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

In [55]:
end_time = Sys.time()
time_matrix[3,]=end_time-start_time

In [56]:
end_time-start_time

Time difference of 2.536521 mins

### The results of the two methods of determining labels are averaged respectively

In [57]:
thetafallmin=(thetafxgbcls_min+thetaflgbcls_min+thetafxgbreg_min+thetaflgbreg_min)/4
thetafallmean=(thetafxgbcls_mean+thetaflgbcls_mean+thetafxgbreg_mean+thetaflgbreg_mean)/4

In [58]:
thetafallminres=predres_mul(thetafallmin,data,thetaf_pr_min,lengthtest,h,testindex)
thetafallmeanres=predres_mul(thetafallmean,data,thetaf_pr_mean,lengthtest,h,testindex)

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”


### Under the above classification, the improved forecast results of the classification and regression models are averaged respectively

In [59]:
thetafclsmin=(thetafxgbcls_min+thetaflgbcls_min)/2
thetafregmin=(thetafxgbreg_min+thetaflgbreg_min)/2
thetafclsmean=(thetafxgbcls_mean+thetaflgbcls_mean)/2
thetafregmean=(thetafxgbreg_mean+thetaflgbreg_mean)/2

In [60]:
thetafclsminres=predres_mul(thetafclsmin,data,thetaf_pr_min[,c(1,3)],lengthtest,h,testindex)
thetafregminres=predres_mul(thetafregmin,data,thetaf_pr_min[,c(2,4)],lengthtest,h,testindex)
thetafclsmeanres=predres_mul(thetafclsmean,data,thetaf_pr_mean[,c(1,3)],lengthtest,h,testindex)
thetafregmeanres=predres_mul(thetafregmean,data,thetaf_pr_mean[,c(2,4)],lengthtest,h,testindex)

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

### Random slice

In [61]:
thetafpred_change<- aperm(thetaf_predh[1:datalength,1:m,1:n,1:h], c(1, 3, 2, 4))  
# 使用array函数将其转换为新的形状
thetafpred_change <- array(thetafpred_change, dim = c(datalength, m*n, h))

In [62]:
model=ets(data[[1]]$x)
fore_l=forecast(model,h=h)
thetafrandomres=matrix(0,datalength,6)
for (i in 1:datalength){
    random=sample(m*n,n)
    minloc=min(random)
    # 使用cut函数将数据分割为区间
    loc <- cut(minloc, breaks =seq(0,m*n,n), labels = FALSE)
    loc<-as.numeric(loc)
    thetafrandompred=apply(thetafpred_change[i,random,],2,mean)
    start=length5[i,loc]
    y_all=data[[i]]$x
    y=y_all[start:length(y_all)]
    fore_l$x=ts(y,frequency=freq,end=end(y_all))
    real=data[[i]]$xx
    fore_l$mean=ts(thetafrandompred,start=start(real),frequency=freq)
    res=accuracy(fore_l,real)
    thetafrandomres[i,]=res[2,1:6]
    }

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute

Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sample”
Warning message in .cbind.ts(list(e1, e2), c(deparse(substitute(e1))[1L], deparse(substitute(e2))[1L]), :
“non-intersecting series”
Warning message in trainingaccuracy(object, test, d, D):
“test elements must be within sampl

In [63]:
print(apply(thetaf_pred_res[testindex,1,1,],2,mean1))
print(apply(thetafminxgbclsres,2,mean1))
print(apply(thetafminxgbregres,2,mean1))
print(apply(thetafminlgbclsres,2,mean1))
print(apply(thetafminlgbregres,2,mean1))

[1]  451.0806687 1040.3339887  907.0310182    0.1415953   17.0564216
[6]    3.3962079
[1]  334.2986160 1019.3006824  889.1046723   -0.6624202   17.4425852
[6]    2.5995908
[1]  355.461661 1003.445998  874.476002   -0.862883   16.899245    2.655734
[1]  398.3758846 1030.8337841  898.8701575    0.1635997   17.1753082
[6]    2.9569913
[1]  327.2828372 1006.9468515  878.4527047   -0.9804477   17.1462941
[6]    2.5047350


In [64]:
print(apply(thetafmeanxgbclsres,2,mean1))
print(apply(thetafmeanxgbregres,2,mean1))
print(apply(thetafmeanlgbclsres,2,mean1))
print(apply(thetafmeanlgbregres,2,mean1))

[1] 306.276167 998.211530 870.112516  -1.448695  17.209216   2.504740
[1]  342.2683004 1002.3168171  874.0945320   -0.9384737   16.9069262
[6]    2.6153461
[1]  393.06000015 1025.70339878  894.52413713   -0.07946415   17.15648750
[6]    2.96385067
[1] 319.515931 995.616921 867.846805  -1.221850  16.910143   2.523546


In [65]:
print(apply(thetafallminres,2,mean1))
print(apply(thetafallmeanres,2,mean1))

[1]  353.8547497 1005.7389676  876.4673123   -0.5855378   16.8886998
[6]    2.8684203
[1] 340.2800996 998.0649947 869.8470376  -0.9221207  16.8382365   2.8564703


In [66]:
print(apply(thetafallminres,2,mean1))
print(apply(thetafallmeanres,2,mean1))

[1]  353.8547497 1005.7389676  876.4673123   -0.5855378   16.8886998
[6]    2.8684203
[1] 340.2800996 998.0649947 869.8470376  -0.9221207  16.8382365   2.8564703


In [67]:
print(apply(thetafclsminres,2,mean1))
print(apply(thetafclsmeanres,2,mean1))
print(apply(thetafregminres,2,mean1))
print(apply(thetafregmeanres,2,mean1))

[1]  366.3372503 1016.2165804  885.8997377   -0.2494102   17.0989454
[6]    2.8644163
[1]  349.6680834 1004.5765298  875.5590555   -0.7640796   16.9742861
[6]    2.8433686
[1]  341.3722491 1001.5068998  873.0266015   -0.9216653   16.9057378
[6]    2.6434894
[1] 330.892116 997.206935 869.369239  -1.080162  16.857707   2.615150


In [68]:
print(apply(thetafrandomres[testindex,],2,mean1))

[1]  393.9089303 1018.9514464  888.0571734   -0.2132143   17.0028764
[6]    3.1914349


In [69]:
thetafall=matrix(0,16,6)
thetafall[1,]=apply(thetaf_pred_res[testindex,1,1,],2,mean1)[1:6]
thetafall[2,]=apply(thetafminxgbclsres,2,mean1)
thetafall[3,]=apply(thetafminxgbregres,2,mean1)
thetafall[4,]=apply(thetafminlgbclsres,2,mean1)
thetafall[5,]=apply(thetafminlgbregres,2,mean1)
thetafall[6,]=apply(thetafmeanxgbclsres,2,mean1)
thetafall[7,]=apply(thetafmeanxgbregres,2,mean1)
thetafall[8,]=apply(thetafmeanlgbclsres,2,mean1)
thetafall[9,]=apply(thetafmeanlgbregres,2,mean1)
thetafall[10,]=apply(thetafallminres,2,mean1)
thetafall[11,]=apply(thetafallmeanres,2,mean1)
thetafall[12,]=apply(thetafclsminres,2,mean1)
thetafall[13,]=apply(thetafclsmeanres,2,mean1)
thetafall[14,]=apply(thetafregminres,2,mean1)
thetafall[15,]=apply(thetafregmeanres,2,mean1)
thetafall[16,]=apply(thetafrandomres[testindex,],2,mean1)

In [70]:
thetafall

451.0807,1040.3340,907.0310,0.14159526,17.05642,3.396208
334.2986,1019.3007,889.1047,-0.66242021,17.44259,2.599591
355.4617,1003.4460,874.4760,-0.86288298,16.89925,2.655734
398.3759,1030.8338,898.8702,0.16359974,17.17531,2.956991
327.2828,1006.9469,878.4527,-0.98044769,17.14629,2.504735
306.2762,998.2115,870.1125,-1.44869507,17.20922,2.504740
342.2683,1002.3168,874.0945,-0.93847372,16.90693,2.615346
393.0600,1025.7034,894.5241,-0.07946415,17.15649,2.963851
319.5159,995.6169,867.8468,-1.22184979,16.91014,2.523546
353.8547,1005.7390,876.4673,-0.58553779,16.88870,2.868420
340.2801,998.0650,869.8470,-0.92212068,16.83824,2.856470


In [71]:
write.csv(thetafall,'thetaf_Yearly_final_res.csv')

In [72]:
time_matrix

user_time,system_time,elapsed_time
1.349246,1.349246,1.349246
2.501386,2.501386,2.501386
2.536521,2.536521,2.536521


In [73]:
write.csv(time_matrix,'Yearly_time_matrix.csv')